In [14]:
import urllib3
from bs4 import BeautifulSoup
import requests
import certifi
import time
import datetime

In [15]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     #passwd="xpx",  # your password
                     db="xudatabase")        # name of the data base

cur = db.cursor()

In [16]:
dailyMax = 1000
batchNum = 20
sleeptimeforPubmed =5
backstep =20

cur.execute('select pmid from last_pmid')
endingPMID = max(cur.fetchall())
startingPMID=endingPMID[0] -backstep #



In [17]:
cur.execute('truncate articles')
# erase data or re-create articles table

0L

In [11]:
%%time
for i in range (startingPMID, startingPMID+dailyMax, batchNum):
    PMIDs = ""
    
    for j in range (batchNum):
        PMIDs = PMIDs + str(i + j ) +","
        
    url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 
    
    #time.sleep(sleeptimeforPubmed)
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, "lxml")
       
    allInfo = unicode(soup.pre.string)
    
    if allInfo.find('PMID- ')<0:
        print url
        print 'bad end'        
        break
    
    while allInfo.find('AB  -')>0:
        IdIndexStart = allInfo.index('PMID- ')+6
        IdIndexEnd = allInfo.index('\n', IdIndexStart)
        PMID = allInfo[IdIndexStart:IdIndexEnd]
    
        abIndexStart = allInfo.index('AB  -')+6
        abIndexEnd = allInfo.index('\n', abIndexStart)
        ab = allInfo[abIndexStart:abIndexEnd]
    
        query = "insert into articles values ("+ PMID +", '"+ ab + "')"
        endingPMID = int(PMID)
        if (endingPMID%10==0):
            print endingPMID

#         try:
#             cur.execute(query)
#             db.commit()

#         except:
#             db.rollback()
           
        allInfo = allInfo[abIndexEnd:]
        
now = datetime.datetime.now()       
query = 'insert into last_pmid values ("%s", %d)'%(now, endingPMID)
cur.execute(query)
db.commit()
# write endingPMID to db
db.close()    
    

28286300
28286310
28286320
28286330
28286340
28286350
28286360
28286370
28286390
28286400
28286410
28286420
28286440
28286450
28286460
28286470
28286480
28286490
28286500
28286510
28286520
28286530
28286540
28286550
28286560
28286570
28286580
28286600
28286610
28286620
28286630
28286640
28286650
28286660
28286670
28286680
28286690
28286700
28286710
28286720
28286730
28286740
28286750
28286760
28286770
28286780
28286790
28286800
28286810
28286820
28286830
28286840
28286850
28286860
28286870
28286880
28286890
28286900
28286910
28286930
28286940
28286950
28286960
28286970
28286980
28286990
28287000
28287010
28287020
28287030
28287050
28287060
28287070
28287080
28287100
28287120
28287130
28287140
28287150
28287160
28287170
28287180
28287190
28287210
28287220
28287230
28287240
28287250
28287260
28287270
28287280
28287290
CPU times: user 1.2 s, sys: 130 ms, total: 1.33 s
Wall time: 37.9 s
